In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm

# Зшивання панорам

Вашим завданням буде реалізація функцій і методів, які складають основні кроки в алгоритмі зшивання панорами із двох зображень. Нагадаю, що цей алгоритм в нашому випадку складається із наступних кроків.

1. **\[SIFT\]** Екстракція ознак і ключових точок SIFT із вхідних зображень. Робимо це на ч/б зображеннях.
2. **\[Matching\]** Шукаємо пари ключових точок, що співпадають.
3. **\[Homography Estimation\]** Оцінюємо проєктивне перетворення із площини зображення 1 на площину зображення 2. Робимо це за допомогою RANSAC + CLS.
4. **\[Perspective Warp\]** Застосовуємо проєктивне перетворення (гомографію) до першого зображення.
5. **\[Blending\]** Склеюємо зображення, змішуємо частини зображень, що перетинаються.

# Важливо!
Результати на різних стадіях для порівнянь можна подивитись в папці `stitching_stages/`

## Вхідні зображення

Ви можете використовувати будь-які зображення, із великим перетином для зшивання. Однак, із даним завданням ідуть фото Золотих Воріт, які рекомендовано використовувати для дебагінгу і тестування.

In [ ]:
left_img_rgb = cv2.imread('media/golden_gate_1.jpg')[..., ::-1]  # read and convert BGR->RGB
left_img_gray = cv2.cvtColor(left_img_rgb, cv2.COLOR_RGB2GRAY)  # convert RGB->Gray

right_img_rgb = cv2.imread('media/golden_gate_2.jpg')[..., ::-1]  # read and convert BGR->RGB
right_img_gray = cv2.cvtColor(right_img_rgb, cv2.COLOR_RGB2GRAY)  # convert RGB->Gray

In [ ]:
def draw_multiple_images(nrows, ncols, images, figsize=(5, 7), dpi=300):
    """Draw a grid of images. """
    assert nrows > 1 or ncols > 1
    f = plt.figure(dpi=dpi, figsize=figsize)
    ax = f.subplots(nrows=nrows, ncols=ncols)
    f.tight_layout()
    if nrows == 1:
        ax = ax[None, ...]
    if ncols == 1:
        ax = ax[..., None]
    for i in range(nrows):
        for j in range(ncols):
            img = images[i][j]
            if len(img.shape) == 2 or min(img.shape) == 1:
                ax[i, j].imshow(img, cmap='gray')
            else:
                ax[i, j].imshow(img)
            ax[i, j].axis('off')
    f.subplots_adjust(wspace=0, hspace=0)

In [ ]:
draw_multiple_images(
    nrows=2, ncols=2,
    images=[[left_img_rgb, right_img_rgb],
            [left_img_gray, right_img_gray]]
)
# stitching_stages/0_input.png

# Крок 1 - SIFT

Екстракція ознак і ключових точок SIFT із вхідних зображень. Робимо це на ч/б (`_gray`) зображеннях.

In [ ]:
def SIFT(img):
    siftDetector = cv2.SIFT_create()
    kp, des = siftDetector.detectAndCompute(img, None)
    return kp, des

def plot_sift(gray, rgb, kp):
    tmp = rgb.copy()
    img = cv2.drawKeypoints(gray, kp, tmp, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return img

Використайте `SIFT` із `OpenCV` - [Посилання на документацію](https://docs.opencv.org/4.x/da/df5/tutorial_py_sift_intro.html) 

In [ ]:
# Запишіть ключові точки та дескриптори в наступні змінні!
left_keypoints, left_descriptors = None, None
right_keypoints, right_descriptors = None, None

In [ ]:
assert left_keypoints is not None and \
       left_descriptors is not None and \
       right_keypoints is not None and \
       right_descriptors is not None, "Заповніть ці змінні!"

## Візуалізуємо отримані ключові точки

In [ ]:
left_sift_picture = cv2.drawKeypoints(
    left_img_gray,
    left_keypoints,
    left_img_rgb.copy(),  # background
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
right_sift_picture = cv2.drawKeypoints(
    right_img_gray,
    right_keypoints,
    right_img_rgb.copy(),  # background
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
draw_multiple_images(
    nrows=1, ncols=2,
    images=[[left_sift_picture, right_sift_picture]]
)
# stitching_stages/1_sifts.png

# Крок 2 - Matching

Шукаємо пари ключових точок, що співпадають.

### Відомості BFMatcher (knnMatch)
Тепер вам потрібно використати BFMatcher і його knnMatch для реалізації матчінгу з К найближчих сусідів. Ось як він працює.

Уявімо собі, що у нас є дві множини чисел - `[9, 50]` та `[10, 51, 55]`.
Тепер поставимо собі задачу знайти пари найближчих чисел. 
`knnMatch(set1, set2, k=2)` поверне множину матчів. Кожен матч для `k=2` буде складатись із двох пар чисел - топ 1 найближчий і топ 2 найближчий. Тобто результат буде приблизно наступним.
`[пара(9, 10), пара(9, 51)], [пара(50, 51), пара(50, 55)]`

Кожна така пара є обʼєктом класу `DMatch`. В ньому є 4 поля - відстань в парі (в нашому випадку просто абсолютна різниця між числами) `distance`, індекс числа в першій множині - `queryIdx`, індекс числа (його пари) з другої множини - `trainIdx`, індекс зображення (для нас не дуже важливий зараз) `imgIdx`.

Приклад використання знаходиться в клітинці нижче. Він ніяк не повʼязаний із завданням, просто слугує для пояснення інтерфейсу.

#### Ви можете реалізувати матчінг з KNN самостійно, але OpenCV варіант є рекомендованим.

In [ ]:
bf = cv2.BFMatcher()

set1 = np.array([9, 50], dtype=np.float32)[..., None]
set2 = np.array([0, 51, 55], dtype=np.float32)[..., None]

matches = bf.knnMatch(set1, set2, k=2)

def print_dmatch(m):
    print(
        f"top1_match=(distance={m.distance}"
        f", queryIdx={m.queryIdx} (num={set1[m.queryIdx][0]})"
        f", trainIdx={m.trainIdx} (num={set2[m.trainIdx][0]})"
        f", imgIdx={m.imgIdx})"
    )

for i_match, match in enumerate(matches):
    print(f"Match #{i_match}")
    top1_match, top2_match = match
    print_dmatch(top1_match)
    print_dmatch(top2_match)

In [ ]:
def match_features(
    left_keypoints, left_descriptors,
    right_keypoints, right_descriptors,
    ratio_test_threshold
):
    # Крок 2.1 - Використайте BFMatcher із knnMatch (OpenCV)
    matches = None

    # Крок 2.2 - Застосуйте ratio test 
    # top_1_match_distance / top_2_match_distance < ratio_test_threshold
    filtered_matches = []

    # Крок 2.3 - складіть ключові точки, що відповідають дескрипторам, які ми матчили вище.
    # Вони лежать в left_keypoints[idx_left].pt
    # і в right_keypoints[idx_right].pt відповідно.
    left_keypoints_matched = []  # масив відповідних ключових точок з лівого зображення.
    right_keypoints_matched = []  # масив відповідних ключових точок з правого зображення.

    return np.array(left_keypoints_matched), np.array(right_keypoints_matched)

In [ ]:
left_keypoints_matched, right_keypoints_matched = match_features(
    left_keypoints, left_descriptors,
    right_keypoints, right_descriptors,
    ratio_test_threshold=0.5
)

In [ ]:
def draw_matches(
    left_keypoints_matched, left_img_rgb,
    right_keypoints_matched, right_img_rgb,
):
    cat_img = np.concatenate((left_img_rgb, right_img_rgb), axis=1)
    fig = plt.figure(dpi=300, figsize=(15, 17))
    ax = fig.subplots()
    ax.set_aspect('equal')
    ax.imshow(cat_img)
    
    right_image_start_x = cat_img.shape[1] / 2
    left_x = left_keypoints_matched[:, 0]
    left_y = left_keypoints_matched[:, 1]
    right_x = right_keypoints_matched[:, 0] + right_image_start_x
    right_y = right_keypoints_matched[:, 1]
    ax.plot([left_x, right_x], [left_y, right_y], 'r', linewidth=0.5, marker='x')
    ax.axis("off")

In [ ]:
draw_matches(left_keypoints_matched, left_img_rgb,
             right_keypoints_matched, right_img_rgb)
# stitching_stages/2_matches.png

# Крок 3 - Homography Estimation

Оцінюємо проєктивне перетворення із площини зображення 1 на площину зображення 2. Робимо це за допомогою RANSAC + CLS.

## Теоретичні відомості 

Матриця гомографії (проєктивного перетворення з площини першого зображення на площину другого) виглядає наступним чином.

$$
\textbf{H} = \begin{bmatrix}
h_{11} & h_{12} & h_{13} \\
h_{21} & h_{22} & h_{23} \\
h_{31} & h_{32} & h_{33}
\end{bmatrix}
$$

Запишемо її коефіцієнти у вигляді довгого вектора.

$$
\textbf{h} = \begin{bmatrix}
h_{11} \\ h_{12} \\ h_{13} \\
h_{21} \\ h_{22} \\ h_{23} \\
h_{31} \\ h_{32} \\ h_{33}
\end{bmatrix}
$$

Нагадаю, що тепер нам потрібно розв'язувати наступну систему рівнянь. В ній, p - ключова точка лівого зображення, q - правого. N - кількість спарованих ключових точок.

$$
A \textbf{h} = \textbf{0}
$$
$$
\begin{bmatrix}
0 & 0 & 0 & p^1_x & p^1_y & 1 & -q^1_y p^1_x & -q^1_y p^1_y & -q^1_y \\
p^1_x & p^1_y & 1 & 0 & 0 & 0 & -q^1_x p^1_x & -q^1_x p^1_y & -q^1_x \\
 \dots &  \dots &  \dots &  \dots &  \dots &  \dots &  \dots &  \dots &  \dots \\
0 & 0 & 0 & p^i_x & p^i_y & 1 & -q^i_y p^i_x & -q^i_y p^i_y & -q^i_y \\
p^i_x & p^i_y & 1 & 0 & 0 & 0 & -q^i_x p^i_x & -q^i_x p^i_y & -q^i_x \\
 \dots &  \dots &  \dots &  \dots &  \dots &  \dots &  \dots &  \dots &  \dots \\
 0 & 0 & 0 & p^N_x & p^N_y & 1 & -q^N_y p^N_x & -q^N_y p^N_y & -q^N_y \\
p^N_x & p^N_y & 1 & 0 & 0 & 0 & -q^N_x p^N_x & -q^N_x p^N_y & -q^N_x \\
\end{bmatrix}
\begin{bmatrix}
h_{11} \\ h_{12} \\ h_{13} \\
h_{21} \\ h_{22} \\ h_{23} \\
h_{31} \\ h_{32} \\ h_{33}
\end{bmatrix} = \textbf{0}
$$

Ми ж переформулюємо цю задачу на задачу мінімізації (CLS):

$$ \textbf{h} = arg\min_{\textbf{h}} ||A\textbf{h}||_2^2 $$

Для цього можна використати модуль `np.linalg`. Погугліть (швидше), або перегляньте лекцію про оцінку параметрів камери.

Зверніть увагу, матриця H має 8 ступенів свободи та 9 коефіцієнтів (проєктивні матриці еквівалентні з точністю до множника). Таким чином ми можемо нормувати матрицю по останньому коефіцієнту. В результаті, шукана матриця гомографії дорівнює:

$$
\textbf{H} = \begin{bmatrix}
\frac{h_{11}}{h_{33}} & \frac{h_{12}}{h_{33}} & \frac{h_{13}}{h_{33}} \\
\frac{h_{21}}{h_{33}} & \frac{h_{22}}{h_{33}} & \frac{h_{23}}{h_{33}} \\
\frac{h_{31}}{h_{33}} & \frac{h_{32}}{h_{33}} & 1
\end{bmatrix}
$$

In [ ]:
def estimate_homography(keypoints1, keypoints2):
    # Побудуйте матрицю A та вирішіть CLS ||Ah||->min
    H = None
    
    # Нормуйте H по останньому к-ту (див відомості).
    H = H / H[2, 2]
    return H

In [ ]:
estimate_homography(left_keypoints_matched, right_keypoints_matched)

## RANSAC

На жаль, ще рано використовувати цю гомографію. Причина в тому, що оцінена матриця буде достатньо чутливою до викидів, які цілком імовірні серед пар ключових точок. Тому, ми будемо використовувати RANSAC.

In [ ]:
def ransac(left_keypoints_matched, right_keypoints_matched, k=5, threshold=0.5, num_iters=2323):
    num_best_inliers = 0

    def sample_random_keypoints(left_keypoints_matched, right_keypoints_matched, k):
        # Візьміть К випадкових чисел від 0 до N-1 - це
        # і будуть індекси наших ключових точок.
        return None

    def compute_projection_error(left_keypoints, right_keypoints, H):
        # Перевіряємо ранг матриці спочатку
        if np.linalg.matrix_rank(H) < 3:
            return np.inf

        # Будь ласка, памʼятайте, що до ключових 2д точок треба додати 1 в 3 вимір.
        # Таким чином [p_x, p_y] -> [p_x, p_y, 1]
        # ------------
        # Підрахунок похибки для пари точок p = left_keypoints[i], q = right_keypoints[i]
        # 1. p = [p_x, p_y, 1]^T
        # 2. p' = Hp
        # 3. p' = [p'_x / p'_z, p'_y / p'_z]^T  (Гомогенізація)
        # 4. error = ||p' - q||^2
        # Поверніть масив похибок для кожної пари точок
        # ------------
        return None
    
    for i in range(num_iters):
        # Вибрати K випадкові пари ключових точок.
        left_keypoints_rand, right_keypoints_rand = sample_random_keypoints(
            left_keypoints_matched, right_keypoints_matched, k=4)

        # оцінюємо гомографію за цими точками
        H = estimate_homography(left_keypoints_rand, right_keypoints_rand)

        # обчисліть похибку проєктування
        errors = compute_projection_error(left_keypoints_matched, right_keypoints_matched, H)
        idx = np.where(errors < threshold)[0]
        left_keypoints_inliers = left_keypoints_matched[idx]
        right_keypoints_inliers = right_keypoints_matched[idx]

        # Якщо кількість інлаєрів більше найкращої, зберігаємо її
        num_inliers = len(left_keypoints_inliers)
        if num_inliers > num_best_inliers:
            left_keypoints_inliers_best = left_keypoints_inliers.copy()
            right_keypoints_inliers_best = right_keypoints_inliers.copy()
            num_best_inliers = num_inliers
            best_H = H.copy()

    mean_best_error = compute_projection_error(left_keypoints_matched, right_keypoints_matched, best_H).mean()
    
    print(f"Кількість інлаєрів = {num_best_inliers}")
    print(f"Всього точок = {len(left_keypoints_matched)}")
    print(f"Гомографія = {best_H}")
    print(f"Середня похибка = {mean_best_error**0.5:.4f} (пікс.)")

    return left_keypoints_inliers_best, right_keypoints_inliers_best, best_H

In [ ]:
left_keypoints_filtered, right_keypoints_filtered, H = ransac(left_keypoints_matched, right_keypoints_matched)

In [ ]:
draw_matches(left_keypoints_filtered, left_img_rgb,
             right_keypoints_filtered, right_img_rgb)
# stitching_stages/3_homography.png

# Крок 4 - Perspective Warp

Застосовуємо проєктивне перетворення (гомографію) до першого зображення.
На цьому кроці ви на самоті. Дам декілька порад.

1 - Використовуйте cv2.warpPerspective.

2 - Єдине, над чим доведеться попрацювати - розміри зображень, додаткова трансляція правих зображень вправо і тд.

3 - Для наступного кроку застосовуйте warpPerspective не тільки до зображень, а й до альфа маски!

In [ ]:
def get_alpha_mask(h, w):
    i = np.arange(h)
    j = np.arange(w)
    j, i = np.meshgrid(j, i)
    
    h_band_width = 0.15
    w_band_width = 0.35
    
    top_band = (i < h_band_width*h) * ((i) / (h_band_width*h) ) + (i >= h_band_width*h)
    bot_band = (i > h-h_band_width*h) * ((h-i-1) / (h_band_width*h) ) + (i <= h-h_band_width*h) 
    
    left_band = (j < w_band_width*w) * ((j) / (w_band_width*w) ) + (j >= w_band_width*w)
    right_band = (j > w-w_band_width*w) * ((w-j-1) / (w_band_width*w) ) + (j <= w-w_band_width*w)
    return top_band*bot_band*right_band*left_band

## Alpha mask

Альфа маска виглядає наступним чином. Вона нам буде потрібна для блендінгу. 
Просто повторюйте з масками ті самі дії, що і з відповідними зображеннями,
потім ми їх використовуватимемо як ваги для пікселів.

In [ ]:
img = get_alpha_mask(*left_img_gray.shape)
plt.imshow(img, cmap='gray')
plt.axis("off");

## Warping!

In [ ]:
def warp_images(left, right, H):
    # Convert to double and normalize. Avoid noise.
    left = cv2.normalize(left.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)   
    # Convert to double and normalize.
    right = cv2.normalize(right.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    left_mask = get_alpha_mask(*left.shape[:2])
    right_mask = get_alpha_mask(*right.shape[:2])

    # Використайте warpPerspective для обох зображень
    # Памʼятайте, що результовані зображення повинні лежати на "спільній" площині
    # великого розміру.
   
    ## warped_l = cv2.warpPerspective(src=left, M=H, dsize=combined_size??)
    ## warped_l_w = cv2.warpPerspective(src=left_mask, M=H, dsize=combined_size??)

    ## warped_r = cv2.warpPerspective(src=right, M=Hr??, dsize=combined_size??)
    ## warped_r_w = cv2.warpPerspective(src=right_mask, M=Hr??, dsize=combined_size??)

    return warped_l, warped_r, warped_l_w, warped_r_w

In [ ]:
left_img_warped, right_img_warped, left_mask_warped, right_mask_warped = warp_images(
    left_img_rgb, right_img_rgb, H)

## Візуалізуємо

In [ ]:
draw_multiple_images(
    nrows=2, ncols=2,
    images=[[left_img_warped, right_img_warped],
            [left_mask_warped, right_mask_warped]],
    figsize=(20, 10)
)

In [ ]:
simple_mask = (left_mask_warped > 0)[..., None]
resulting_image = left_img_warped * simple_mask + right_img_warped * (1 - simple_mask)
fig = plt.figure()
plt.imshow(resulting_image, cmap='gray');
plt.axis('off');
# stitching_stages/4_simple_stitching.png

# Крок 5 - Blending

Як бачите, видно шви. Щоб їх прибрати, застосуємо блендінг.

Склеюємо зображення, змішуємо частини зображень, що перетинаються.

In [ ]:
simple_mask_left = left_mask_warped > 0
simple_mask_right = right_mask_warped > 0

# Спочатку присвоїмо вагам, які не лежать на перетині зображень вагу 1.
right_mask_warped[~simple_mask_left] = (right_mask_warped[~simple_mask_left] > 0).astype(float)
left_mask_warped[~simple_mask_right] = (left_mask_warped[~simple_mask_right] > 0).astype(float)

# Підрахуємо маску перетину.
intersection_mask = simple_mask_left & simple_mask_right

# Ваги будуть дорівнювати W1/(W1 + W2) та W2/(W1 + W2) відповідно.
w_total = left_mask_warped[intersection_mask] + right_mask_warped[intersection_mask]
left_mask_warped[intersection_mask] /= w_total
right_mask_warped[intersection_mask] /= w_total

# Накладаємо зображення із вагами.
resulting_image = left_img_warped * left_mask_warped[..., None] + right_img_warped * right_mask_warped[..., None]
plt.figure(dpi=300)
plt.imshow(resulting_image, cmap='gray')
plt.axis("off");

# stitching_stages/5_blending.png

# Вітаю! Ви завершили завдання

# Excellency project:
Зшивач панорам із більш ніж двох зображень. Зшити два зображення достатньо просто, але не завжди! Не завжди точки бувають робастними, не завжди альфа-блендінг простий, зображення ще треба кропнути, а шви не завжди є лінійними! Звертайтесь, якщо вам це цікаво.